# Modèle de Random Forest

## Importation de modules

In [1]:
import numpy as np
import pandas as pd
from math import *
import matplotlib.pyplot as plt
import scipy

In [ ]:
## Chargement et formatage des données

In [2]:
df= pd.read_parquet("/Users/sirinelouati/Desktop/dataset2.parquet", engine="pyarrow")

In [3]:
def transform_parquet_to_csv(df,name_csv):
    #df=pd.read_parquet(file_path, engine="pyarrow")
    ind=df.index
    columns1=[]
    for i in range (1,257):
      col1= 'audio_feature_%i' % i
      columns1.append(col1)
    columns2=[]
    for i in range (1,129):
      col2= 'usage_feature_%i' % i
      columns2.append(col2)
    split_df_audio = pd.DataFrame(df['audio_features'].tolist(),columns=columns1,index=ind)
    split_df_usage=pd.DataFrame(df['usage_features'].tolist(),columns=columns2,index=ind)
    df1 = pd.concat([df, split_df_audio], axis=1)
    df =pd.concat([df1, split_df_usage], axis=1)
    df.drop(columns=['audio_features','usage_features'], inplace=True)
    df.set_index('song_index', inplace=True,drop=True)
    df.to_csv(name_csv)
    return df

In [4]:
transform_parquet_to_csv(df,'dataset.csv')

,asian,rnb,reggae,blues,pop,dance,folk,arabic-music,indie,rock,...,usage_feature_119,usage_feature_120,usage_feature_121,usage_feature_122,usage_feature_123,usage_feature_124,usage_feature_125,usage_feature_126,usage_feature_127,usage_feature_128
song_index,,,,,,,,,,,,,,,,,,,,,
195,0,0,0,0,0,0,0,0,0,0,...,0.008582,-0.000024,-0.272207,0.210503,0.030972,0.080964,-0.015423,0.102148,0.365081,-0.066897
417,0,0,0,0,0,0,0,0,0,1,...,-0.182309,-0.051225,-0.004617,-0.005726,-0.038334,-0.063972,0.053881,-0.001152,0.045032,0.027505
530,0,0,0,0,1,0,0,0,0,0,...,-0.044620,-0.132282,-0.141684,-0.106271,-0.189065,-0.229804,0.083824,-0.160790,0.040417,-0.094503
909,0,0,0,0,0,0,0,0,0,1,...,-0.127573,-0.068515,-0.106374,-0.102694,-0.066937,0.026911,-0.032158,0.063670,-0.026653,-0.068251
1257,0,0,0,0,0,0,0,0,0,0,...,0.024463,-0.052952,-0.059056,0.041670,-0.025673,-0.000446,-0.003438,0.060359,0.058938,0.029523
1312,0,0,0,0,0,0,0,0,0,0,...,0.033834,0.059702,0.127839,0.042218,-0.058068,-0.132771,0.048156,0.002076,0.139599,0.032241
1538,0,0,0,0,1,0,0,0,0,0,...,0.005377,-0.009041,-0.023785,-0.019660,0.013792,0.036384,-0.066490,0.040814,-0.007791,-0.038196
1819,0,0,0,0,0,0,0,0,0,1,...,0.079662,-0.017283,0.060251,0.050107,0.106449,-0.074171,0.099201,-0.030231,-0.114429,-0.007822
1878,0,0,0,0,0,0,0,0,0,0,...,-0.017524,-0.030789,-0.035952,-0.053677,-0.015703,0.001386,0.011345,0.022860,0.039047,-0.013739


In [5]:
df=pd.read_csv('dataset.csv')

In [10]:
df.isnull().any()

song_index           False
asian                False
rnb                  False
reggae               False
blues                False
pop                  False
dance                False
folk                 False
arabic-music         False
indie                False
rock                 False
soulfunk             False
latin                False
classical            False
k-pop                False
brazilian            False
metal                False
rap                  False
jazz                 False
electronic           False
african              False
country              False
artist_name          False
song_title           False
audio_feature_1      False
audio_feature_2      False
audio_feature_3      False
audio_feature_4      False
audio_feature_5      False
audio_feature_6      False
                     ...  
usage_feature_99     False
usage_feature_100    False
usage_feature_101    False
usage_feature_102    False
usage_feature_103    False
usage_feature_104    False
u

Nous créons le vecteur de labels Y et le vecteur des variables explicatives X

In [6]:
def extract_values_array(df,p,usage=False):
    n=df.shape[0]
    df_extracted= df.sample(int(p*n), random_state=0)
    df_extracted.drop(columns=['song_title','artist_name'],inplace=True)
    #pour qu'on ait des sorties similaires à travers plusieurs appels
    columns3=[]
    if (usage==False):
        for i in range (1,129):
            col2= 'usage_feature_%i' % i
            columns3.append(col2)
        df_extracted.drop(columns=columns3,inplace=True)
    l=list(df_extracted.columns[0:22])
    x=list(df_extracted.columns[22:])
    l.remove('song_index')
    X=df_extracted[x].values
    Y=df_extracted[l].values
    return(X,Y)

In [7]:
X,Y= extract_values_array(df,0.1)

In [13]:
df.head(5)

,song_index,asian,rnb,reggae,blues,pop,dance,folk,arabic-music,indie,...,usage_feature_119,usage_feature_120,usage_feature_121,usage_feature_122,usage_feature_123,usage_feature_124,usage_feature_125,usage_feature_126,usage_feature_127,usage_feature_128
0,195,0,0,0,0,0,0,0,0,0,...,0.008582,-0.000024,-0.272207,0.210503,0.030972,0.080964,-0.015423,0.102148,0.365081,-0.066897
1,417,0,0,0,0,0,0,0,0,0,...,-0.182309,-0.051225,-0.004617,-0.005726,-0.038334,-0.063972,0.053881,-0.001152,0.045032,0.027505
2,530,0,0,0,0,1,0,0,0,0,...,-0.044620,-0.132282,-0.141684,-0.106271,-0.189065,-0.229804,0.083824,-0.160790,0.040417,-0.094503
3,909,0,0,0,0,0,0,0,0,0,...,-0.127573,-0.068515,-0.106374,-0.102694,-0.066937,0.026911,-0.032158,0.063670,-0.026653,-0.068251
4,1257,0,0,0,0,0,0,0,0,0,...,0.024463,-0.052952,-0.059056,0.041670,-0.025673,-0.000446,-0.003438,0.060359,0.058938,0.029523


## Importation de modules pour le Random Forest

In [8]:
import pandas as pd # Pour le dataframe
import numpy as np # Pour la normalisation et calculs de moyenne
import matplotlib.pyplot as plt # Pour la visualisation

from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve, RandomizedSearchCV 
# Split de dataset et optimisation des hyperparamètres
from sklearn.ensemble import RandomForestClassifier # Random forest

from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score, zero_one_loss, classification_report 
# Métriques pour la mesure de performances

On affiche tous les paramètres qui peuvent être optimisés

In [9]:
rf = RandomForestClassifier(random_state = 0)
from pprint import pprint
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}


On affiche la grille à tester. (1ère tentative)

In [23]:
 # nombre d'arbres
n_estimators = [ 4000, 5000]
# profondeur max de l'arbre
max_depth = [20]
max_depth.append(None)
# nombre d'échantillon min nécessaire par noeuds
min_samples_split = [2, 4]#[2]
# nombre d'échantillon min nécessaire par feuilles
min_samples_leaf = [1, 2]#[1]

In [24]:
# création de la grille
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              }
pprint(random_grid)

{'max_depth': [20, None],
 'min_samples_leaf': [1, 2],
 'min_samples_split': [2, 4],
 'n_estimators': [4000, 5000]}


### Création du modèle de Random Forest

In [10]:
rf = RandomForestClassifier(random_state = 0, max_features = 'sqrt', bootstrap = True)

In [1]:
# random search
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=0, n_jobs = -1)

In [12]:
from sklearn.preprocessing import normalize, StandardScaler

In [13]:
# séparer les données en training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(X, Y, test_size = 0.25, random_state = 0)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

sc = StandardScaler()
train_features = sc.fit_transform(train_features)
test_features = sc.transform(test_features)

Training Features Shape: (3792, 256)
Training Labels Shape: (3792, 21)
Testing Features Shape: (1264, 256)
Testing Labels Shape: (1264, 21)


### Optimisation des hyperparamètres

In [25]:
# création du modèle
rf = RandomForestClassifier(random_state = 0, max_features = 'sqrt', bootstrap = True)

# random search
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, cv = 3, verbose=2, random_state=0, n_jobs = None)

# fit le modèle
rf_random.fit(train_features, train_labels)

pd_res = pd.concat([pd.DataFrame(rf_random.cv_results_["params"]),pd.DataFrame(rf_random.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1)
pd_res = pd_res.sort_values('Accuracy', ascending=False)
print(rf_random.best_params_)
pd_res.head(5)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] n_estimators=5000, min_samples_split=2, min_samples_leaf=1, max_depth=20 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=5000, min_samples_split=2, min_samples_leaf=1, max_depth=20, total= 7.1min
[CV] n_estimators=5000, min_samples_split=2, min_samples_leaf=1, max_depth=20 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.5min remaining:    0.0s


[CV]  n_estimators=5000, min_samples_split=2, min_samples_leaf=1, max_depth=20, total= 6.9min
[CV] n_estimators=5000, min_samples_split=2, min_samples_leaf=1, max_depth=20 
[CV]  n_estimators=5000, min_samples_split=2, min_samples_leaf=1, max_depth=20, total= 6.9min
[CV] n_estimators=4000, min_samples_split=4, min_samples_leaf=2, max_depth=20 
[CV]  n_estimators=4000, min_samples_split=4, min_samples_leaf=2, max_depth=20, total= 5.0min
[CV] n_estimators=4000, min_samples_split=4, min_samples_leaf=2, max_depth=20 
[CV]  n_estimators=4000, min_samples_split=4, min_samples_leaf=2, max_depth=20, total= 4.9min
[CV] n_estimators=4000, min_samples_split=4, min_samples_leaf=2, max_depth=20 
[CV]  n_estimators=4000, min_samples_split=4, min_samples_leaf=2, max_depth=20, total= 5.0min
[CV] n_estimators=4000, min_samples_split=2, min_samples_leaf=1, max_depth=None 
[CV]  n_estimators=4000, min_samples_split=2, min_samples_leaf=1, max_depth=None, total= 5.6min
[CV] n_estimators=4000, min_samples_s

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 99.4min finished


{'n_estimators': 5000, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': None}


,max_depth,min_samples_leaf,min_samples_split,n_estimators,Accuracy
4,NaN,2,2,5000,0.232331
0,20.0,1,2,5000,0.232068
1,20.0,2,4,4000,0.231804
2,NaN,1,2,4000,0.231540
3,NaN,1,2,5000,0.230485


On affiche la grille à tester. (tentative finale)

In [14]:
# nombre d'arbres
n_estimators = [2000, 3000,4000, 5000]
# profondeur max de l'arbre
max_depth = [20]
max_depth.append(None)
# nombre d'échantillon min nécessaire par noeuds
min_samples_split = [2, 4]#[2]
# nombre d'échantillon min nécessaire par feuilles
min_samples_leaf = [1, 2]#[1]

In [15]:
# création de la grille
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              }
pprint(random_grid)

{'max_depth': [20, None],
 'min_samples_leaf': [1, 2],
 'min_samples_split': [2, 4],
 'n_estimators': [2000, 3000, 4000, 5000]}


In [16]:
# création du modèle
rf = RandomForestClassifier(random_state = 0, max_features = 'sqrt', bootstrap = True)

# random search
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, cv = 3, verbose=2, random_state=0, n_jobs = 1)

# fit le modèle
rf_random.fit(train_features, train_labels)

pd_res = pd.concat([pd.DataFrame(rf_random.cv_results_["params"]),pd.DataFrame(rf_random.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1)
pd_res = pd_res.sort_values('Accuracy', ascending=False)
print(rf_random.best_params_)
pd_res.head(5)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] n_estimators=5000, min_samples_split=2, min_samples_leaf=2, max_depth=20 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=5000, min_samples_split=2, min_samples_leaf=2, max_depth=20, total= 6.9min
[CV] n_estimators=5000, min_samples_split=2, min_samples_leaf=2, max_depth=20 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.3min remaining:    0.0s


[CV]  n_estimators=5000, min_samples_split=2, min_samples_leaf=2, max_depth=20, total= 5.9min
[CV] n_estimators=5000, min_samples_split=2, min_samples_leaf=2, max_depth=20 
[CV]  n_estimators=5000, min_samples_split=2, min_samples_leaf=2, max_depth=20, total= 5.9min
[CV] n_estimators=4000, min_samples_split=4, min_samples_leaf=1, max_depth=None 
[CV]  n_estimators=4000, min_samples_split=4, min_samples_leaf=1, max_depth=None, total= 5.1min
[CV] n_estimators=4000, min_samples_split=4, min_samples_leaf=1, max_depth=None 
[CV]  n_estimators=4000, min_samples_split=4, min_samples_leaf=1, max_depth=None, total= 5.1min
[CV] n_estimators=4000, min_samples_split=4, min_samples_leaf=1, max_depth=None 
[CV]  n_estimators=4000, min_samples_split=4, min_samples_leaf=1, max_depth=None, total= 5.2min
[CV] n_estimators=4000, min_samples_split=2, min_samples_leaf=2, max_depth=20 
[CV]  n_estimators=4000, min_samples_split=2, min_samples_leaf=2, max_depth=20, total= 4.7min
[CV] n_estimators=4000, min_s

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 76.4min finished


{'n_estimators': 4000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20}


,max_depth,min_samples_leaf,min_samples_split,n_estimators,Accuracy
3,20.0,1,2,4000,0.232859
0,20.0,2,2,5000,0.232331
2,20.0,2,2,4000,0.231804
4,NaN,1,2,2000,0.231276
1,NaN,1,4,4000,0.231013


### Métriques du Random Forest

In [17]:
# création du modèle
rf = RandomForestClassifier(n_estimators=4000, max_features='sqrt', max_depth=20, min_samples_split=2, min_samples_leaf=1, bootstrap=True, criterion='gini' ,random_state=0)

# fit le modèle
rf.fit(train_features, train_labels)

# prédictions
predictions = rf.predict(test_features)

# Zero_one_loss error
errors = zero_one_loss(test_labels, predictions, normalize=False)
print('zero_one_loss error :', errors)

# Accuracy Score
accuracy_test = accuracy_score(test_labels, predictions)
print('accuracy_score on test dataset :', accuracy_test)

print(classification_report(predictions, test_labels))

zero_one_loss error : 983
accuracy_score on test dataset : 0.22231012658227847
              precision    recall  f1-score   support

           0       0.06      1.00      0.11         1
           1       0.00      0.00      0.00         0
           2       0.02      1.00      0.04         1
           3       0.00      0.00      0.00         0
           4       0.04      0.70      0.08        10
           5       0.19      0.83      0.31        24
           6       0.00      0.00      0.00         0
           7       0.01      1.00      0.03         1
           8       0.00      0.00      0.00         0
           9       0.47      0.70      0.56       138
          10       0.00      0.00      0.00         0
          11       0.06      1.00      0.11         5
          12       0.56      0.82      0.67        11
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.71      0.89      0.79       111
  

/Users/sirinelouati/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/sirinelouati/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
